In [0]:
import pandas as pd
from plotly import graph_objs as go
import plotly.express as px
import numpy as np
pd.set_option('mode.chained_assignment', None)

In [8]:
#READ DATA IN FROM GITHUB
#deaths=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')

#new deaths file from JHU
deaths=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

#UNPIVOT INTO A LONG DF and RENAME COLUMNS
deaths2 = pd.melt(deaths,id_vars=['Province/State','Country/Region','Lat','Long'])
deaths2.rename(columns={"variable": "date", "value": "deaths"},inplace=True)

#SET DATE DATATYPE
deaths2['date'] = pd.to_datetime(deaths2['date'])
deaths_agg = pd.DataFrame(deaths2.groupby(['Country/Region','date'])['deaths'].sum()).reset_index()

#LIMIT TO COUNTRIES OF INTEREST
countries_of_interest = ['US','China','Italy','Spain','Iran','France','Korea, South','Japan','United Kingdom']
final = deaths_agg[deaths_agg['Country/Region'].isin(countries_of_interest)]

#GET FIRST DATE WHERE DEATHS > 10 and START COUNTING FROM THERE
final['d10'] = np.where(final['deaths']>= 10,1,0)
final['days_since_10']=final.groupby(by=['Country/Region'])['d10'].cumsum()

#LIMIT TO ONLY FIRST 35 DAYS SINCE 10 DEATHS
final_use = final[final['days_since_10']<=35]

print("The most recent day is", np.max(final_use['date']))

The most recent day is 2020-03-27 00:00:00


In [27]:
print(final_use)

       Country/Region       date  deaths  d10  days_since_10  recent
2244            China 2020-01-22      17    1              1       0
2245            China 2020-01-23      18    1              2       0
2246            China 2020-01-24      26    1              3       0
2247            China 2020-01-25      42    1              4       0
2248            China 2020-01-26      56    1              5       0
...               ...        ...     ...  ...            ...     ...
11149  United Kingdom 2020-03-23     336    1             10       0
11150  United Kingdom 2020-03-24     423    1             11       0
11151  United Kingdom 2020-03-25     466    1             12       0
11152  United Kingdom 2020-03-26     580    1             13       0
11153  United Kingdom 2020-03-27     761    1             14       1

[563 rows x 6 columns]


In [152]:
#PLOT USING PLOTLY, LOOP OVER COUNTRIES TO GET ANNOTATION POINTS
fig = go.Figure()
annotations = []
i = 0

for c in countries_of_interest:
    color = px.colors.qualitative.G10[i]

    annotations.append(go.layout.Annotation(
        x = final_use[final_use['Country/Region']=='{}'.format(c)].iloc[-1]['days_since_10'],
        y = final_use[final_use['Country/Region']=='{}'.format(c)].iloc[-1]['deaths'],
        text=c,
        font=dict(
            family="Franklin Gothic",
            size=16,
            color="black"
            ))
        )

    #make the data in a bad way
    tmp = final_use[final_use['Country/Region'].isin([c])]
    y1= tmp['deaths']
    x1 = tmp['days_since_10']

    #this lets us control the lines
    fig.add_trace(go.Scatter(x=x1, y=y1, name = c, mode='lines+markers', line=dict(color = color, width=3)))
    i += 1

fig.update(layout=go.Layout(showlegend=False, annotations=annotations, height=500, width=1000))
fig.update_layout(
    title={
      'text': "COVID-19 tracker: Deaths since 10th death",
      'y':0.9,
      'x':0.5,
      'xanchor': 'center',
      'yanchor': 'top'},
    xaxis_title="Days since 10th death"
)

max = np.max(final_use['date'])
note = 'Updated ' + str(max.month) + '-' + str(max.day)
fig.add_annotation(
        x=5,
        y=9000,
        xref="x",
        yref="y",
        text=note,
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="red"
            ),
        align="center",
        ax=20,
        ay=-30,
        bordercolor="black",
        borderwidth=2,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
        )

fig.add_annotation(
    x=5,
    y=8000,
    text="Source: JHU",
    showarrow=False,
    font=dict(
      family="Courier New, monospace",
      size=16,
      color="red"
      ),
)


#print(fig.layout)
fig.layout.template = 'plotly_white'
fig.layout.template = 'presentation'

fig.show()
